In [ ]:
!pip install -q scikit-image scipy

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import Dataset
import torch
import numpy as np, cv2, math, random, os, glob, csv
import matplotlib.pyplot as plt
from PIL import Image
from skimage.morphology import skeletonize
from scipy.optimize import curve_fit
from scipy.ndimage import median_filter
from skimage.morphology import skeletonize
from google.colab import files
from tqdm import tqdm
import joblib

In [ ]:
!unzip tiles_images.zip

Archive:  tiles_images.zip
   creating: tiles_images/
   creating: tiles_images/images/
  inflating: tiles_images/images/image_0038.png  
  inflating: tiles_images/images/image_0046.png  
  inflating: tiles_images/images/image_0048.png  
 extracting: tiles_images/images/image_0050.png  
 extracting: tiles_images/images/image_0052.png  
 extracting: tiles_images/images/image_0054.png  
  inflating: tiles_images/images/image_0056.png  
 extracting: tiles_images/images/image_0058.png  
  inflating: tiles_images/images/image_0060.png  
  inflating: tiles_images/images/image_0064.png  
 extracting: tiles_images/images/image_0066.png  
 extracting: tiles_images/images/image_0068.png  
  inflating: tiles_images/images/image_0070.png  
  inflating: tiles_images/images/image_0072.png  
 extracting: tiles_images/images/image_0074.png  
  inflating: tiles_images/images/image_0078.png  
 extracting: tiles_images/images/image_0086.png  
 extracting: tiles_images/images/image_0088.png  
  inflating:

In [ ]:
class CurveDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.imgs = sorted(glob.glob(os.path.join(img_dir, '*.png')))
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = Image.open(img_path).convert('RGB')
        img = np.array(img)

        if self.transform:
            img = self.transform(image=img)['image']
        return img

In [ ]:
ds = CurveDataset("/content/tiles_images/images")

In [ ]:
ω = None

def contours_from_mask(mask, min_len=50, min_span_ratio=0.1):
    h, w = mask.shape
    cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    good=[]
    for c in cnts:
        if len(c) < min_len:
          continue
        if np.ptp(c[:,0,0]) / w < min_span_ratio:
          continue
        good.append(c[:,0,:])
    return good

def fit_sinusoid(points, ω, h, guess=None, max_pts=2000):
    points = np.asarray(points)
    xs, ys = points[:, 0], points[:, 1]

    if len(xs) > max_pts:
        sel = np.random.choice(len(xs), max_pts, replace=False)
        xs, ys = xs[sel], ys[sel]

    order = np.argsort(xs)
    xs, ys = xs[order], ys[order]

    ys = median_filter(ys, size=5)

    if guess is None:
        guess = (np.ptp(ys)/2, 0.0, ys.mean())

    def model(x, A, θ, y0):
        return A * np.sin(ω * x + θ) + y0

    try:
        A, θ, y0 = curve_fit(model, xs, ys, p0=guess, maxfev=5000)[0]
        if A < 0:
            A = -A
            θ = (θ + math.pi) % (2 * math.pi)
        return A, θ, y0
    except RuntimeError:
        return None

def split_upper_lower(pts: np.ndarray, min_x_span: int = 50):
    xs, ys = pts[:, 0], pts[:, 1]
    uniq_x = np.unique(xs)
    if np.ptp(uniq_x) < min_x_span:
        return []
    upper, lower = [], []
    for x in uniq_x:
        ys_here = ys[xs == x]
        upper.append((x, ys_here.min()))
        lower.append((x, ys_here.max()))
    return [np.array(upper), np.array(lower)]

def treshold(img_rgb, low_thresh=0.1, high_thresh=0.6):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    thresh_img = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 25, 15)
    blurred = cv2.GaussianBlur(thresh_img, (5,5), 1.41)
    edges = cv2.Canny(blurred, int(low_thresh * 255), int(high_thresh*255), apertureSize=3, L2gradient=True)
    edges = cv2.dilate(edges, (3, 3), iterations=1)
    return edges

def preprocess_light(img_rgb, low_thresh=0.1, high_thresh=0.6):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    _, thresh_img = cv2.threshold(gray, 0, 255, cv2.THRESH_TOZERO + cv2.THRESH_OTSU)
    blurred = cv2.GaussianBlur(thresh_img, (5,5), 1.41)
    edges = cv2.Canny(blurred, int(low_thresh * 255), int(high_thresh*255), apertureSize=3, L2gradient=True)
    edges = cv2.dilate(edges, (3, 3), iterations=1)
    return edges

def preprocess(img_rgb, low_thresh=0.1, high_thresh=0.6):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    _, thresh_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blurred = cv2.GaussianBlur(gray, (5,5), 1.41)
    edges = cv2.Canny(blurred, int(low_thresh * 255), int(high_thresh*255), apertureSize=3, L2gradient=True)
    edges = cv2.dilate(edges, (3, 3), iterations=1)
    return edges

def preprocess_edges_dark_light_version(img_rgb, low_thresh=0.1, high_thresh=0.6):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    _, thresh_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blurred = cv2.GaussianBlur(thresh_img, (5,5), 1.41)
    edges = cv2.Canny(blurred, int(low_thresh * 255), int(high_thresh*255), apertureSize=3, L2gradient=True)
    edges = cv2.dilate(edges, (3, 3), iterations=1)
    return edges

def preprocess_edges_dark(img_rgb, low_thresh=0.1, high_thresh=0.6):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (5,5), 1.41)
    edges = cv2.Canny(blurred, int(low_thresh * 255), int(high_thresh*255), apertureSize=3, L2gradient=True)
    edges = cv2.dilate(edges, (3, 3), iterations=1)
    return edges

def detect_and_draw(img_rgb,
                    background="rgb",
                    top_N=3,
                    rms_thr=10.0,
                    min_len=40,
                    span_ratio=0.03,
                    show_debug=False):

    h, w = img_rgb.shape[:2]
    ω = 2 * np.pi / w

    edges_dark = preprocess_edges_dark(img_rgb)
    edges_light = preprocess_edges_dark_light_version(img_rgb)
    edges = preprocess(img_rgb)
    edges_second = preprocess_light(img_rgb)
    edges_tr = treshold (img_rgb)

    contours_dark = contours_from_mask(edges_dark, min_len, span_ratio)
    contours_light = contours_from_mask(edges_light, min_len, span_ratio)
    contours = contours_from_mask(edges, min_len, span_ratio)
    contours_second = contours_from_mask(edges_second, min_len, span_ratio)
    contours_tr = contours_from_mask (edges_tr, min_len, span_ratio)

    scored = []

    for cnt in contours_dark:
        for pts in split_upper_lower(cnt, min_x_span=30):
            if len(pts) < 30:
                continue

            fit = fit_sinusoid(pts, ω, h)
            if fit is None:
                continue
            A, θ, y0 = fit

            if A < 2 or A > h / 1.5:
                continue

            xs, ys = pts[:, 0], pts[:, 1]
            rmse = np.sqrt(((ys - (A*np.sin(ω*xs + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue

            geom = len(pts) / rmse
            if A > 200:
              continue

            scored.append((geom, A, θ, y0))

    for cnt in contours_light:
        for pts in split_upper_lower(cnt, min_x_span=30):
            if len(pts) < 30:
                continue

            fit = fit_sinusoid(pts, ω, h)
            if fit is None:
                continue
            A, θ, y0 = fit

            if A < 2 or A > h / 1.5:
                continue

            xs, ys = pts[:, 0], pts[:, 1]
            rmse = np.sqrt(((ys - (A*np.sin(ω*xs + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue

            geom = len(pts) / rmse
            if A > 200:
                continue
            scored.append((geom, A, θ, y0))

    for cnt in contours:
        for pts in split_upper_lower(cnt, min_x_span=30):
            if len(pts) < 30:
                continue

            fit = fit_sinusoid(pts, ω, h)
            if fit is None:
                continue
            A, θ, y0 = fit

            if A < 2 or A > h / 1.5:
                continue

            xs, ys = pts[:, 0], pts[:, 1]
            rmse = np.sqrt(((ys - (A*np.sin(ω*xs + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue

            geom = len(pts) / rmse
            if A > 200:
                continue
            scored.append((geom, A, θ, y0))

    for cnt in contours_tr:
        for pts in split_upper_lower(cnt, min_x_span=30):
            if len(pts) < 30:
                continue

            fit = fit_sinusoid(pts, ω, h)
            if fit is None:
                continue
            A, θ, y0 = fit

            if A < 2 or A > h / 1.5:
                continue

            xs, ys = pts[:, 0], pts[:, 1]
            rmse = np.sqrt(((ys - (A*np.sin(ω*xs + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue

            geom = len(pts) / rmse
            if A > 200:
                continue
            scored.append((geom, A, θ, y0))

    scored.sort(key=lambda t: t[0], reverse=True)
    curves = [(A, θ, y0) for _, A, θ, y0 in scored[:top_N]]

    canvas_1 = cv2.cvtColor(edges_dark, cv2.COLOR_GRAY2BGR) if background == "mask" else img_rgb.copy()
    canvas_2 = cv2.cvtColor(edges_light, cv2.COLOR_GRAY2BGR) if background == "mask" else img_rgb.copy()
    canvas_3 = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR) if background == "mask" else img_rgb.copy()
    canvas_4 = cv2.cvtColor(edges_second, cv2.COLOR_GRAY2BGR) if background == "mask" else img_rgb.copy()
    canvas_5 = cv2.cvtColor(edges_tr, cv2.COLOR_GRAY2BGR) if background == "mask" else img_rgb.copy()

    xs_full = np.arange(w)
    for A, θ, y0 in curves:
        ys = (A * np.sin(ω*xs_full + θ) + y0).astype(int)
        good = (ys >= 0) & (ys < h)
        pts = np.column_stack([xs_full[good], ys[good]]).astype(np.int32)
        cv2.polylines(canvas_1, [pts], False, (0, 255, 0), 3)

    for A, θ, y0 in curves:
        ys = (A * np.sin(ω*xs_full + θ) + y0).astype(int)
        good = (ys >= 0) & (ys < h)
        pts = np.column_stack([xs_full[good], ys[good]]).astype(np.int32)
        cv2.polylines(canvas_2, [pts], False, (0, 255, 0), 3)

    for A, θ, y0 in curves:
        ys = (A * np.sin(ω*xs_full + θ) + y0).astype(int)
        good = (ys >= 0) & (ys < h)
        pts = np.column_stack([xs_full[good], ys[good]]).astype(np.int32)
        cv2.polylines(canvas_3, [pts], False, (0, 255, 0), 3)

    for A, θ, y0 in curves:
        ys = (A * np.sin(ω*xs_full + θ) + y0).astype(int)
        good = (ys >= 0) & (ys < h)
        pts = np.column_stack([xs_full[good], ys[good]]).astype(np.int32)
        cv2.polylines(canvas_4, [pts], False, (0, 255, 0), 3)

    for A, θ, y0 in curves:
        ys = (A * np.sin(ω*xs_full + θ) + y0).astype(int)
        good = (ys >= 0) & (ys < h)
        pts = np.column_stack([xs_full[good], ys[good]]).astype(np.int32)
        cv2.polylines(canvas_5, [pts], False, (0, 255, 0), 3)

    plt.figure(figsize=(6, 6))

    plt.imshow(canvas_1)
    plt.imshow(canvas_2)
    plt.imshow(canvas_3)
    plt.imshow(canvas_4)
    plt.imshow(canvas_5)

    plt.axis("off")
    plt.title(f"{len(curves)} штуки")
    plt.show()

    for c in contours:
        plt.plot(c[:, 0], c[:, 1], '.', markersize=1)
    plt.gca().invert_yaxis()
    plt.title("Контуры до fit_sin")
    plt.show()

    if show_debug:
        plt.figure(figsize=(4, 4))

        plt.imshow(edges_dark, cmap="gray")
        plt.imshow(edges_light, cmap="gray")
        plt.imshow(edges, cmap="gray")
        plt.imshow(edges_second, cmap="gray")
        plt.imshow(edges_tr, cmap="gray")

        plt.axis("off")
        plt.title("edges")
        plt.show()


for i in range (45, 56):
    sample_path = ds.imgs[i]
    img = np.array(Image.open(sample_path).convert('RGB'))
    detect_and_draw(img_rgb=img, show_debug=True)

sample_path = ds.imgs[100]
img = np.array(Image.open(sample_path).convert('RGB'))
detect_and_draw(img_rgb=img, show_debug=True)


In [ ]:
def draw_sine(img, A, θ, y0, ω, color=(0, 255, 0), thickness=3):
    h, w = img.shape[:2]
    xs = np.arange(w)
    ys = (A * np.sin(ω * xs + θ) + y0).astype(int)
    mask = (ys >= 0) & (ys < h)
    pts = np.column_stack([xs[mask], ys[mask]]).astype(np.int32)
    if len(pts) > 1:
        cv2.polylines(img, [pts], isClosed=False, color=color, thickness=thickness)

In [ ]:
model_csv = "/content/forest_model.pkl"

In [ ]:
saved = joblib.load(model_csv)
clf = saved['model']
scaler = saved['scaler']

In [ ]:
IMG_PATH = "/content/val.jpg"

img = np.array(Image.open(IMG_PATH).convert("RGB"))
h, w = img.shape[:2]
ω = 2 * math.pi / w

up, down = 540.351, 718.725
PIX2MM = (down - up) / h * 1000

rows, rows_for_pred, xs = [], [], np.arange(w)
out = img.copy()

step = 256
stride = 64
max_per_tile = 4
min_dist_x = w // (max_per_tile + 1)
min_dist_y = 40

for top in tqdm(range(0, h, stride)):
    bottom  = min(h, top + step)
    crop    = img[top:bottom]

    crop_h  = crop.shape[0]

    edges_dark = preprocess_edges_dark(crop)
    edges_light = preprocess_edges_dark_light_version(crop)
    edges_third = preprocess(crop)
    edges = preprocess_light(crop)

    contours_dark = contours_from_mask(edges_dark)
    contours_light = contours_from_mask(edges_light)
    contours_third = contours_from_mask(edges_third)
    contours = contours_from_mask (edges)

    cand = []
    for pts in contours_dark:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours_light:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours_third:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))


    cand.sort(reverse=True)
    selected, xs_used, y_used = [], [], []

    for _, A_px, A_mm, θ, y0_global, x0 in cand:
        if len(selected) >= max_per_tile:
            break
        if all(abs(x0 - xu) > min_dist_x for xu in xs_used) and all(abs(y0_global - yu) > min_dist_y for yu in y_used):
            xs_used.append(x0)
            y_used.append(y0_global)
            selected.append((A_px, A_mm, θ, y0_global))

    for A_px, A_mm, θ, y0_global in selected:
        depth_m = up + (down - up) * (y0_global / h)

        rows.append([depth_m, A_mm, θ])
        rows_for_pred.append([depth_m, A_mm, θ, A_px, y0_global])

df_rows = pd.DataFrame(rows_for_pred, columns=["depth_m", "A_mm", "theta_rad", "A_px", "y0_global"])
X_pred_scaled = scaler.transform(df_rows[["depth_m", "A_mm", "theta_rad"]])
y_pred = clf.predict(X_pred_scaled)

out = img.copy()
filtered = [(row, row_pred) for row, row_pred in zip(rows_for_pred, y_pred) if row_pred == 1]
filtered_rows = [row[:3] for row, _ in filtered]

for (depth_m, A_mm, θ, A_px, y0_global), _ in filtered:
    ys = (A_px * np.sin(ω * xs + θ) + y0_global).astype(int)
    good = (ys >= 0) & (ys < h)
    pts_vis = np.column_stack([xs[good], ys[good]]).astype(np.int32)
    cv2.polylines(out, [pts_vis], False, (0, 255, 0), 2)

rows_sorted = sorted(filtered_rows, key=lambda r: r[0])
seen = set()
unique_rows = []
for r in rows_sorted:
    depth = round(r[0], 4)
    if depth not in seen:
        seen.add(depth)
        unique_rows.append(r)

print (len(unique_rows))

  0%|          | 0/552 [00:00<?, ?it/s]/tmp/ipython-input-101-1148012441.py:35: OptimizeWarning: Covariance of the parameters could not be estimated
  A, θ, y0 = curve_fit(model, xs, ys, p0=guess, maxfev=5000)[0]
100%|██████████| 552/552 [02:31<00:00,  3.64it/s]


337


In [ ]:
IMG_PATH = "/content/val.jpg"

rows, rows_for_pred, xs = [], [], np.arange(w)
out = img.copy()

for top in tqdm(range(0, h, stride)):
    bottom  = min(h, top + step)
    crop    = img[top:bottom]

    crop_h  = crop.shape[0]

    edges_dark = preprocess_edges_dark(crop)
    edges_light = preprocess_edges_dark_light_version(crop)
    edges_third = preprocess(crop)
    edges = preprocess_light(crop)

    contours_dark = contours_from_mask(edges_dark)
    contours_light = contours_from_mask(edges_light)
    contours_third = contours_from_mask(edges_third)
    contours = contours_from_mask(edges)

    cand = []
    for pts in contours_dark:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours_light:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours_third:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    cand.sort(reverse=True)
    selected, xs_used, y_used = [], [], []

    for _, A_px, A_mm, θ, y0_global, x0 in cand:
        if len(selected) >= max_per_tile:
            break
        if all(abs(x0 - xu) > min_dist_x for xu in xs_used) and all(abs(y0_global - yu) > min_dist_y for yu in y_used):
            xs_used.append(x0)
            y_used.append(y0_global)
            selected.append((A_px, A_mm, θ, y0_global))

    for A_px, A_mm, θ, y0_global in selected:
        depth_m = up + (down - up) * (y0_global / h)

        rows.append([depth_m, A_mm, θ])
        rows_for_pred.append([depth_m, A_mm, θ, A_px, y0_global])

df_rows = pd.DataFrame(rows_for_pred, columns=["depth_m", "A_mm", "theta_rad", "A_px", "y0_global"])
X_pred_scaled = scaler.transform(df_rows[["depth_m", "A_mm", "theta_rad"]])
y_pred = clf.predict(X_pred_scaled)

out = img.copy()
filtered = [(row, row_pred) for row, row_pred in zip(rows_for_pred, y_pred) if row_pred == 1]
filtered_rows = [row[:3] for row, _ in filtered]

for (depth_m, A_mm, θ, A_px, y0_global), _ in filtered:
    ys = (A_px * np.sin(ω * xs + θ) + y0_global).astype(int)
    good = (ys >= 0) & (ys < h)
    pts_vis = np.column_stack([xs[good], ys[good]]).astype(np.int32)
    cv2.polylines(out, [pts_vis], False, (0, 255, 0), 2)

rows_sorted = sorted(filtered_rows, key=lambda r: r[0])
seen = set()
for r in rows_sorted:
    depth = round(r[0], 4)
    if depth not in seen:
        seen.add(depth)
        unique_rows.append(r)

print (len(unique_rows))

  0%|          | 0/552 [00:00<?, ?it/s]/tmp/ipython-input-101-1148012441.py:35: OptimizeWarning: Covariance of the parameters could not be estimated
  A, θ, y0 = curve_fit(model, xs, ys, p0=guess, maxfev=5000)[0]
100%|██████████| 552/552 [02:19<00:00,  3.95it/s]


674


In [ ]:
IMG_PATH = "/content/val.jpg"

rows, rows_for_pred, xs = [], [], np.arange(w)
out = img.copy()

for top in tqdm(range(0, h, stride)):
    bottom  = min(h, top + step)
    crop    = img[top:bottom]

    crop_h  = crop.shape[0]

    edges_dark = preprocess_edges_dark(crop)
    edges_light = preprocess_edges_dark_light_version(crop)
    edges_third = preprocess(crop)
    edges = preprocess_light(crop)

    edges_tr = treshold(crop)

    contours_dark = contours_from_mask(edges_dark)
    contours_light = contours_from_mask(edges_light)
    contours_third = contours_from_mask(edges_third)
    contours = contours_from_mask (edges)

    contours_tr = contours_from_mask(edges_tr)

    cand = []
    for pts in contours_dark:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours_light:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours_third:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    for pts in contours_tr:
        for branch in split_upper_lower(pts, 30):
            if branch.shape[0] < 30:
                continue
            res = fit_sinusoid(branch, ω, crop_h)
            if not res:
                continue
            A_px, θ, y0 = res
            A_mm = A_px * PIX2MM
            if A_mm > 200 or A_px < 2 or A_px > crop_h / 1.5:
                continue
            rmse = np.sqrt(((branch[:,1] - (A_px * np.sin(ω * branch[:,0] + θ) + y0)) ** 2).mean())
            if rmse > 10:
                continue
            x0 = branch[:,0].min()
            x1 = branch[:,0].max()
            d0 = A_px * ω * math.cos(ω * x0 + θ)
            d1 = A_px * ω * math.cos(ω * x1 + θ)
            if d0 * d1 <= 0:
                continue
            score = len(branch) / rmse
            cand.append((score, A_px, A_mm, θ, y0 + top, x0))

    cand.sort(reverse=True)
    selected, xs_used, y_used = [], [], []

    for _, A_px, A_mm, θ, y0_global, x0 in cand:
        if len(selected) >= max_per_tile:
            break
        if all(abs(x0 - xu) > min_dist_x for xu in xs_used) and all(abs(y0_global - yu) > min_dist_y for yu in y_used):
            xs_used.append(x0)
            y_used.append(y0_global)
            selected.append((A_px, A_mm, θ, y0_global))

    for A_px, A_mm, θ, y0_global in selected:
        depth_m = up + (down - up) * (y0_global / h)

        rows.append([depth_m, A_mm, θ])
        rows_for_pred.append([depth_m, A_mm, θ, A_px, y0_global])

df_rows = pd.DataFrame(rows_for_pred, columns=["depth_m", "A_mm", "theta_rad", "A_px", "y0_global"])
X_pred_scaled = scaler.transform(df_rows[["depth_m", "A_mm", "theta_rad"]])
y_pred = clf.predict(X_pred_scaled)

out = img.copy()
filtered = [(row, row_pred) for row, row_pred in zip(rows_for_pred, y_pred) if row_pred == 1]
filtered_rows = [row[:3] for row, _ in filtered]

for (depth_m, A_mm, θ, A_px, y0_global), _ in filtered:
    ys = (A_px * np.sin(ω * xs + θ) + y0_global).astype(int)
    good = (ys >= 0) & (ys < h)
    pts_vis = np.column_stack([xs[good], ys[good]]).astype(np.int32)
    cv2.polylines(out, [pts_vis], False, (0, 255, 0), 2)

rows_sorted = sorted(filtered_rows, key=lambda r: r[0])
seen = set()
unique_rows_tr = []
for r in rows_sorted:
    depth = round(r[0], 4)
    if depth not in seen:
        seen.add(depth)
        unique_rows_tr.append(r)

print (len(unique_rows_tr))

  0%|          | 0/552 [00:00<?, ?it/s]/tmp/ipython-input-101-1148012441.py:35: OptimizeWarning: Covariance of the parameters could not be estimated
  A, θ, y0 = curve_fit(model, xs, ys, p0=guess, maxfev=5000)[0]
100%|██████████| 552/552 [02:48<00:00,  3.28it/s]


336


In [ ]:
CSV_PATH = "/content/result.csv"

THRESHOLD = 0.1

filtered_rows = sorted([r for r in unique_rows if r[1] >= 45], key=lambda r: r[0])

grouped_rows = []
current_group = [filtered_rows[0]]
for row in filtered_rows[1:]:
    if abs(row[0] - current_group[-1][0]) < THRESHOLD:
        current_group.append(row)
    else:
        grouped_rows.append(current_group)
        current_group = [row]
grouped_rows.append(current_group)

averaged_rows = []
depths_used = []

for group in grouped_rows:
    depths = [r[0] for r in group]
    amplitudes = [r[1] for r in group]
    thetas = [r[2] for r in group]

    depth_avg = round(np.mean(depths), 10)
    amp_avg = round(np.mean(amplitudes), 10)

    idx_closest = np.argmin([abs(a - amp_avg) for a in amplitudes])
    theta_selected = thetas[idx_closest]

    averaged_rows.append([depth_avg, amp_avg, theta_selected])
    depths_used.append(depth_avg)

filtered_rows_tr = [
    r for r in unique_rows_tr
    if all(abs(r[0] - d) >= THRESHOLD for d in depths_used)
]

combined_rows = averaged_rows + filtered_rows_tr
combined_rows = sorted(combined_rows, key=lambda r: r[0])

grouped_rows_final = []
current_group = [combined_rows[0]]

for row in combined_rows[1:]:
    if abs(row[0] - current_group[-1][0]) < THRESHOLD:
        current_group.append(row)
    else:
        grouped_rows_final.append(current_group)
        current_group = [row]
grouped_rows_final.append(current_group)

final_averaged_rows = []
for group in grouped_rows_final:
    depths = [r[0] for r in group]
    amplitudes = [r[1] for r in group]
    thetas = [r[2] for r in group]

    depth_avg = round(np.mean(depths), 10)
    amp_avg = round(np.mean(amplitudes), 10)

    idx_closest = np.argmin([abs(a - amp_avg) for a in amplitudes])
    theta_selected = thetas[idx_closest]

    final_averaged_rows.append([depth_avg, amp_avg, theta_selected])

with open(CSV_PATH, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["depth_m", "A_mm", "theta_rad"])
    writer.writerows(final_averaged_rows)

In [ ]:
len(pd.read_csv (CSV_PATH))

305